``` SQL
UPDATE ALIGNEMENTS
SET pointsActuels = (SELECT COUNT(Event) AS NbPTS     
  FROM PBP
       INNER JOIN
       ALIGNEMENTS ON (PBP.p1_ID = ALIGNEMENTS.idNHL) OR 
                      (PBP.p2_ID = ALIGNEMENTS.idNHL) OR 
                      (PBP.p3_ID = ALIGNEMENTS.idNHL) 
 WHERE EVENT = "GOAL" AND 
       statutJoueur = "Alignement" AND 
       PBP.Date > ALIGNEMENTS.dateDebut AND
       PBP.Date <= ALIGNEMENTS.dateFin
GROUP BY idAlignement)
 ```

Une cellule qui:

    - (DONE) (majPoints) qui met à jour les buts, passes et PJ des joueurs selon la période donnée dans Alignements
    - (DONE) qui met à jour le nombre de points de chaque pooler
    - (DONE) qui met à jour la masse salariale de chaque pooler
    - (DONE) qui met à jour les json
    - (testPierre) exécute le sync avec GitHub

In [11]:
#variables globales

conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
fin = " WHERE EVENT = 'GOAL' AND PBP.Date > ALIGNEMENTS.dateDebut AND PBP.Date <= ALIGNEMENTS.dateFin AND PBP.Date > '2019-08-25' GROUP BY idAlignement)) WHERE idAlignement = ALIGNEMENTS.idAlignement)"

#fonctions
def execSQL(sql, conn, write=False):
    c = conn.cursor()
    print(c.execute(sql))
    if write:
        conn.commit()
    else:
        print(c.fetchall())
    c.close()

def writeUpdateDebut(typeScore):
    updateDebut = "UPDATE ALIGNEMENTS SET " + typeScore + "Actuels = (SELECT Nb" + typeScore + " FROM ((SELECT idAlignement, COUNT(Event) AS Nb" + typeScore + " FROM PBP INNER JOIN ALIGNEMENTS ON"
    return updateDebut
    
def majInterne(typeScore):
    debut = writeUpdateDebut(typeScore)
    if typeScore == "buts":
        milieu = " (PBP.p1_ID = ALIGNEMENTS.idNHL)"
    elif typeScore == "assist":
        milieu = " (PBP.p2_ID = ALIGNEMENTS.idNHL) OR (PBP.p3_ID = ALIGNEMENTS.idNHL)"
    sqlUpdate = debut + milieu + fin
    execSQL(sqlUpdate, conn, write = True)

def majPointsAlignements():
    sqlUpdate = "UPDATE ALIGNEMENTS SET pointsActuels = (butsActuels + assistActuels)"
    execSQL(sqlUpdate, conn, write = True)

def majPJ():
    sqlUpdate = "UPDATE ALIGNEMENTS SET pjActuels = (SELECT NbPJ FROM (SELECT idAlignement, COUNT(DISTINCT Game_Id) AS NbPJ FROM SHIFTS INNER JOIN ALIGNEMENTS ON (SHIFTS.Player_Id = ALIGNEMENTS.idNHL) AND SHIFTS.Date > ALIGNEMENTS.dateDebut AND SHIFTS.Date <= ALIGNEMENTS.dateFin AND SHIFTS.Date > '2019-08-25' GROUP BY idAlignement) WHERE idAlignement = ALIGNEMENTS.idAlignement)"
    execSQL(sqlUpdate, conn, write = True)

def majPJBPPAlign():
    majInterne("buts")
    majInterne("assist")
    majPointsAlignements()
    majPJ()

def majScorePoolers():
    sqlUpdate = "UPDATE POOLERS SET PJ = (SELECT NbPJ FROM (SELECT idPooler, SUM(pjActuels) AS NbPJ FROM ALIGNEMENTS GROUP BY idPooler) WHERE Id = idPooler), B = (SELECT NbB FROM (SELECT idPooler, SUM(butsActuels) AS NbB FROM ALIGNEMENTS GROUP BY idPooler) WHERE Id = idPooler), A = (SELECT NbA FROM (SELECT idPooler, SUM(assistActuels) AS NbA FROM ALIGNEMENTS GROUP BY idPooler) WHERE Id = idPooler)"
    sqlUpdate2 = "UPDATE POOLERS SET Score = (POOLERS.B + POOLERS.A)"
    execSQL(sqlUpdate, conn, write = True)
    execSQL(sqlUpdate2, conn, write = True)

#majMasseSalariale()
def majMasseSalariale():
    sqlUpdate = "UPDATE POOLERS SET MasseSalariale = (SELECT masseActuelle FROM (SELECT idPooler, SUM(salaireActuel) AS masseActuelle FROM ALIGNEMENTS INNER JOIN JOUEURS ON ALIGNEMENTS.idNHL = JOUEURS.idNHL WHERE '2019-12-25' BETWEEN ALIGNEMENTS.dateDebut AND ALIGNEMENTS.dateFin GROUP BY idPooler) WHERE Poolers.Id = idPooler)"
    execSQL(sqlUpdate, conn, write = True)

#maj des json
def majTables():
    conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
    var = 'OK'
    try:
        #maj Poolers
        dfPoolers = pd.read_sql("SELECT * FROM POOLERS", conn)
        jsonPoolers = dfPoolers.to_json("C:/dev/pool-de-hockey/src/data/poolers.json", orient = "table", index =    False, indent = 4)
    except Exception as ex:
        print("Erreur : poolers")
        var = "Erreur poolers"

    try:
        #maj Alignements
        requete = "SELECT * FROM ALIGNEMENTS INNER JOIN JOUEURS ON JOUEURS.idNHL = ALIGNEMENTS.idNHL"
        dfTempo = pd.read_sql(requete, conn)
        dfAlignements = dfTempo.T.drop_duplicates().T
        jsonAlignements = dfAlignements.to_json("C:/dev/pool-de-hockey/src/data/alignements.json", orient = "table", index = False, indent = 4)
    except Exception as ex2:
        print("Erreur : alignements")
        var = "Erreur alignements"

    try:
        #maj Joueurs
        dfJoueurs = pd.read_sql("SELECT * FROM JOUEURS", conn)
        jsonJoueurs = dfJoueurs.to_json("C:/dev/pool-de-hockey/src/data/joueurs.json", orient = "table", index = False, indent = 4)
    except Exception as ex3:
        print("Erreur : joueurs")
        var = "Erreur joueurs"

    return var


In [12]:
majPJBPPAlign()

In [13]:
majScorePoolers()

In [14]:
majMasseSalariale()

In [15]:
majTables()

'OK'